In [ ]:
import numpy as np
import pandas as pd
import awkward_pandas
import uproot
import matplotlib.pyplot as plt
import os
import math
import itertools

from scipy.optimize import curve_fit, fsolve
from scipy.special import gamma
from scipy.special import gammaincc

CMTOMICRON = 1e4

In [ ]:
def readRootTrees(runNumber: int) -> dict:
    """
    Reads all trees from a ROOT file and returns them as a dictionary.
    The keys are the tree names and the values are pandas DataFrames.

    Args:
        runNumber (int): The run number of the ROOT file.

    Returns:
        dict: A dictionary where keys are tree names (str) and
              values are pandas DataFrames. Returns an empty dictionary
              if the file cannot be opened or contains no trees.
    """

    dataFilePath = 'Data/'
    dataFile = f'sim.{runNumber}.root'
    fullFilePath = os.path.join(dataFilePath, dataFile)
    
    try:
        with uproot.open(fullFilePath) as rootFile:
            dataframes = {}
            for treeName in rootFile.keys():
                if isinstance(rootFile[treeName], uproot.behaviors.TTree.TTree):
                    tree = rootFile[treeName]
                    try:
                        df = tree.arrays(library='pd')
                        dataframes[treeName] = df
                    except Exception as e:
                        print(f"Error reading tree '{treeName}': {e}")
            return dataframes
    except Exception as e:
        print(f"Error opening or reading ROOT file '{fullFilePath}': {e}")
        return {}

In [ ]:
runNumber = 5252
simData = readRootTrees(runNumber)
for key in simData:
    globals()[key[:-6]] = simData[key]
    print(key[:-6])

In [ ]:
print(metaData.iloc[0])

In [ ]:
def plotCellGeometry(metaData):
    """
    """
    # Extract relevant geometric parameters from metadata.
    pitch = metaData['Pitch'].iloc[0]

    inRadius = pitch/2.
    outRadius = 2*inRadius/math.sqrt(3)
    
    padLength = metaData['Pad Length'].iloc[0]
    
    holeRadius = metaData['Hole Radius'].iloc[0]

    #Centers of neighbouring cells
    pitchX = pitch*2./math.sqrt(3)
    pitchY = inRadius
    neighbourX = 3./2.*outRadius*np.array([-1, -1, 0, 0, 1, 1])
    neighbourY = inRadius*np.array([1, -1, 2, -2, 1, -1])

    #Define the vertices of a hexagon
    hexCornerX = np.array([1., .5, -.5, -1., -.5, .5, 1.])
    hexCornerY = math.sqrt(3.)/2.*np.array([0., 1., 1., 0., -1., -1., 0.])
    
    # Corners of the pad
    padX = hexCornerX*padLength
    padY = hexCornerY*padLength

    # Corners of the cell
    cellX = hexCornerX*outRadius
    cellY = hexCornerY*outRadius

    # Define circles representing the holes in the grid.
    #    (Must be done as separate patches)
    hole = {}
    #Hole for primary cell
    hole[0] = plt.Circle((0, 0), holeRadius, 
                         facecolor='none', edgecolor='k', lw=1, 
                         label=f'Hole (r = {holeRadius*CMTOMICRON:.0f} um)')
    # Neighbouring cell holes (#808080 = Grey)
    for i in range(len(neighbourX)):
        hole[i+1] = plt.Circle((neighbourX[i], neighbourY[i]), holeRadius, 
                               facecolor='none', edgecolor='#808080', ls=':', lw=1)

    # Define circles representing the pillars.
    #    (Must be done as separate patches)
    pillarRadius = .0005
    pillar = {}
    for i in range(6):
        pillar[i] = plt.Circle((outRadius*hexCornerX[i], outRadius*hexCornerY[i]), pillarRadius, 
                               facecolor='none', edgecolor='#808080', ls='--', lw=1)

    pillar[0].set_label(f'Pillar  (r = {pillarRadius*CMTOMICRON:.0f} um)')

    
    # Make figure and add plots
    fig = plt.figure(figsize=(10, 6))
    fig.suptitle(f'Cell Geometry')
    ax1 = fig.add_subplot(111)


    #Add the pad
    ax1.plot(padX, padY, 
             label='Pad', c='m', lw=1)

    #Add the cell boundary
    ax1.plot(cellX, cellY, 
             label='Cell Boundary', c='b', lw=1)
    
    #Add boundaries of neighboring cells and pads
    for i in range(6):
        ax1.plot(neighbourX[i]+cellX, neighbourY[i]+cellY,
                 c='c', ls=':', lw=1)
        ax1.plot(neighbourX[i]+padX, neighbourY[i]+padY,
                 c='m', ls=':', lw=1)
    
    #Add the holes in the grid
    for i in range(len(hole)):
        ax1.add_patch(hole[i])

    #Add pillars
    for i in range(len(pillar)):
        ax1.add_patch(pillar[i])

    #Add markers to center of cells
    ax1.plot(0., 0., marker='x', c='r')
    ax1.plot(neighbourX, neighbourY,
             marker='.', c='r', ls='')

    #Add geometry cell
    geoX = 3./2.*outRadius*np.array([-1, 1, 1, -1, -1])
    geoY = inRadius*np.array([1, 1, -1, -1, 1])
    ax1.plot(geoX, geoY,
             c='g', ls='--', lw=1, label='Simulation Boundary')
    #Possible large geometry that eliminates shared boundaries
    #ax1.plot(geoX, 2*geoY,
    #         c='g', ls=':', lw=1, label='Possible Geometry')

    #Add dimensions
    ax1.plot([0, neighbourX[4]], [0, neighbourY[4]],
             label=f'Pitch ({pitch*CMTOMICRON:.0f} um)', c='r', ls=':', lw=1)
    ax1.plot([0, padLength], [0, 0],
             label=f'Pad Length ({padLength*CMTOMICRON:.0f} um)', c='r', ls='-', lw=1)
    
    # Add grid lines and set aspect
    ax1.grid()
    axLim = 1.1*pitch
    ax1.set_xlim(-axLim, axLim)
    ax1.set_ylim(-axLim, axLim)
    ax1.set_aspect('equal')

    ax1.set_xlabel('x (cm)')
    ax1.set_ylabel('y (cm)')

    ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    return

plotCellGeometry(metaData)

In [ ]:
def plotAddCellGeometry(axis, axes, metaData):
    """
    """
    pitch = metaData['Pitch'].iloc[0]
    inRadius = pitch/2.
    outRadius = 2*inRadius/math.sqrt(3)
    
    padLength = metaData['Pad Length'].iloc[0]

    halfGridThickness = metaData['Grid Thickness'].iloc[0]/2.
    padHeight = -halfGridThickness - metaData['Grid Standoff'].iloc[0]
    cathodeHeight = halfGridThickness + metaData['Cathode Height'].iloc[0]
    holeRadius = metaData['Hole Radius'].iloc[0]

    #Define the vertices of a hexagon
    hexCornerX = np.array([1., .5, -.5, -1., -.5, .5, 1.])
    hexCornerY = math.sqrt(3.)/2.*np.array([0., 1., 1., 0., -1., -1., 0.])
    
    # Corners of the pad
    padX = hexCornerX*padLength
    padY = hexCornerY*padLength
    padZ = np.ones(len(padX))*padHeight

    # Corners of the cell
    cellX = hexCornerX*outRadius
    cellY = hexCornerY*outRadius

    # Define the hole in the grid.
    hole = plt.Circle((0, 0), holeRadius,
                      facecolor='none', edgecolor='k', lw=1, label='Hole')
    holeXY = holeRadius*np.array([-1, -1, 1, 1, -1])
    holeZ = halfGridThickness*np.array([1, 1, -1, -1, 1])

    
    axLim = 1.1*pitch
    
    match axes:
        case 'xy':
            axis.plot(padX, padY, 
                      label='Pad', c='m', lw=1)
            axis.plot(cellX, cellY, 
                      label='Cell', c='c', lw=1)
            axis.add_patch(hole)

            axis.set_xlabel('x (cm)')
            axis.set_ylabel('y (cm)')

            axis.set_xlim(-axLim, axLim)
            axis.set_ylim(-axLim, axLim)
            axis.set_aspect('equal')

        case 'xz':
            axis.plot(padX, padZ, 
                      label='Pad', c='m', lw=2)
            axis.plot([cellX[3], cellX[0], cellX[0], cellX[3], cellX[3]], 
                      [padHeight, padHeight, cathodeHeight, cathodeHeight, padHeight],
                      label='Cell', c='c', lw=1)
            axis.plot([cellX[2], cellX[1], cellX[1], cellX[2], cellX[2]], 
                      [padHeight, padHeight, cathodeHeight, cathodeHeight, padHeight], 
                      c='c', ls='--', lw=1)
            axis.plot(holeXY, holeZ, 
                      label='Hole', c='k', ls='-')
            
            axis.set_xlabel('x (cm)')
            axis.set_ylabel('z (cm)')

            axis.set_xlim(-axLim, axLim)
            axis.set_ylim(padHeight, cathodeHeight)

        case 'yz':
            axis.plot(padY, padZ, 
                      label='Pad', c='m', lw=2)
            axis.plot([cellY[4], cellY[1], cellY[1], cellY[4], cellY[4]], 
                      [padHeight, padHeight, cathodeHeight, cathodeHeight, padHeight],
                      label='Cell', c='c', lw=1)
            axis.plot([0, 0], 
                      [padHeight, cathodeHeight],
                      label='Cell', c='c', ls='--', lw=1)
            axis.plot(holeXY, holeZ, 
                      label='Hole', c='k', ls='-')

            axis.set_xlabel('y (cm)')
            axis.set_ylabel('z (cm)')

            axis.set_xlim(-axLim, axLim)
            axis.set_ylim(padHeight, cathodeHeight)
            

        case _:
            print(f'Invalid form of plot axes: {axes}')

    axis.grid()

    return
    

In [ ]:
def plotFieldLines(fieldLines, metaData, plotName) -> None:
    """
    Generates 2D plots visualizing electric field lines around a pad.

    The plots display the field lines, the pad geometry, and the hole
    in the grid.

    Args:
        fieldLines: A Pandas DataFrame containing the xyz coordinates of the field lines.
        metaData (DataFrame): A DataFrame containing metadata about the detector geometry.
        plotName (str): A string used to title the plots.

    Returns:
        None
    """

    # Group field lines together based on the line ID
    groupedFieldLines = fieldLines.groupby('Field Line ID')

    # Create figure and subplots for different projections.
    fig2D = plt.figure(figsize=(14, 7))
    fig2D.suptitle(f'Field Lines - {plotName}')
    ax11 = fig2D.add_subplot(221)
    ax12 = fig2D.add_subplot(223)
    ax13 = fig2D.add_subplot(122)

    # Iterate through each field line and plot.
    for lineID, fieldLine in groupedFieldLines:
        ax11.plot(fieldLine['Field Line x'],
                  fieldLine['Field Line z'])
        ax12.plot(fieldLine['Field Line y'],
                  fieldLine['Field Line z'])
        ax13.plot(fieldLine['Field Line x'],
                  fieldLine['Field Line y'])

    #Add geometry Pieces and axis labeling
    plotAddCellGeometry(ax11, 'xz', metaData)
    plotAddCellGeometry(ax12, 'yz', metaData)
    plotAddCellGeometry(ax13, 'xy', metaData)

    return

In [ ]:
fieldLines = fieldLineData
plotFieldLines(fieldLines, metaData, 'Cathode')

aboveGridLines = gridFieldLineData[gridFieldLineData['Grid Line Location']==1]
plotFieldLines(aboveGridLines, metaData, 'Above Grid')

belowGridLines = gridFieldLineData[gridFieldLineData['Grid Line Location']==-1]
plotFieldLines(belowGridLines, metaData, 'Below Grid')

In [ ]:
def plotLocations(particleData, metaData, plotName) -> None:
    """
    Generates 2D histograms to visualize the inital and final locations
    of simulated particles from many avalanches.

    The pad geometry and the hole in the grid are included.

    Args:
        particleData (DataFrame): A Pandas DataFrame containing the initial and final
                                  coordinates for a type of particle.
        metaData (DataFrame): A DataFrame containing metadata about the detector geometry.
        plotName (str): A string used to title the plots.

    Returns:
        None
    """
    # Extract relevant geometric parameters from metadata.
    pitch = metaData['Pitch'].iloc[0]
    halfGridThickness = metaData['Grid Thickness'].iloc[0]/2.
    padHeight = -halfGridThickness - metaData['Grid Standoff'].iloc[0]
    cathodeHeight = halfGridThickness + metaData['Cathode Height'].iloc[0]

    numBins = 51

    # Create the figure and add subplots
    fig = plt.figure(figsize=(12, 12))
    fig.suptitle(f'Particle Locations: {plotName}')
    
    ax1 = fig.add_subplot(221)
    ax2 = fig.add_subplot(222)
    ax3 = fig.add_subplot(223)
    ax4 = fig.add_subplot(224)

    rangeXY = [[-pitch, pitch], [-pitch, pitch]]
    rangeXZ = [[-pitch, pitch], [padHeight, cathodeHeight]]

    # Plot data
    ax1.hist2d(particleData['Initial x'], 
               particleData['Initial y'], 
               bins=numBins, range=rangeXY, cmin=1)
    ax2.hist2d(particleData['Final x'], 
               particleData['Final y'], 
               bins=numBins, range=rangeXY, cmin=1)
    ax3.hist2d(particleData['Initial x'], 
               particleData['Initial z'], 
               bins=numBins, range=rangeXZ, cmin=1)
    ax4.hist2d(particleData['Final x'], 
               particleData['Final z'], 
               bins=numBins, range=rangeXZ, cmin=1)

    #Add geometry Pieces
    plotAddCellGeometry(ax1, 'xy', metaData)
    plotAddCellGeometry(ax2, 'xy', metaData)
    plotAddCellGeometry(ax3, 'xz', metaData)
    plotAddCellGeometry(ax4, 'xz', metaData)

    ax1.set_title('Initial')
    ax2.set_title('Final')
    ax3.set_title('Initial')
    ax4.set_title('Final')


    return

In [ ]:
newElectrons = electronData[electronData['Electron ID'] != 0]
plotLocations(newElectrons, metaData, 'Electrons')

positiveIons = ionData[(ionData['Ion Charge'] == 1) & (ionData['Electron ID'] != 0)]
plotLocations(positiveIons, metaData, 'Positive Ions')

negativeIons = ionData[(ionData['Ion Charge'] == -1)]
plotLocations(negativeIons, metaData, 'Negative Ions')

In [ ]:
halfPitch = .99*metaData['Pitch'].iloc[0]/2.
cathodeIons = positiveIons[(positiveIons['Final z'] > 0.039)]
plotLocations(cathodeIons, metaData, f'Cathode Ions ({len(cathodeIons)}/{len(positiveIons)})')

In [ ]:
def plotDiffusion(particleData, metaData, plotName) -> None:
    """
    """
    halfPitch = metaData['Pitch'].iloc[0]/2.
    halfGridThickness = metaData['Grid Thickness'].iloc[0]/2.
    padHeight = -halfGridThickness - metaData['Grid Standoff'].iloc[0]
    cathodeHeight = halfGridThickness + metaData['Cathode Height'].iloc[0]
    
    driftX = particleData['Final x'] - particleData['Initial x']
    driftY = particleData['Final y'] - particleData['Initial y']
    driftZ = particleData['Final z'] - particleData['Initial z']

    driftR = np.sqrt(np.square(driftX) + np.square(driftY))

    fig = plt.figure(figsize=(12, 4))
    fig.suptitle(f'{plotName}')
    
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)

    ax1.hist(abs(driftZ))
    ax2.hist(driftR)

    ax1.set_title('Drift in z')
    ax2.set_title('Drift in r')

    ax1.set_xlabel('Distance in z (cm)')
    ax2.set_xlabel('Distance in xy plane (cm)')

    ax1.grid()
    ax2.grid()
    
    return

In [ ]:
plotDiffusion(electronData, metaData, 'All Electron Diffusion')
plotDiffusion(electronData[electronData['Exit Status'] == -1], metaData, 'Exit Side (Status = -1)')
plotDiffusion(electronData[electronData['Exit Status'] == -5], metaData, 'Exit Bottom  (Status = -5)')
plotDiffusion(electronData[electronData['Exit Status'] == -7], metaData, 'Attached Electrons')

In [ ]:
def plotSingleAvalanche(electronTrackData, avalancheID, metaData, plotName) -> None:
    """
    Plots the drift lines of all electrons from a specified simulated electron avalanche.

    The pad geometry and the hole in the grid are included.

    Args:
        electronTrackData (DataFrame): A Pandas DataFrame containing the xyz coordinates of electron tracks.
        avalancheID (int): The ID number of the desired avalanche to plot
        metaData (DataFrame): A DataFrame containing metadata about the detector geometry.
        plotName (str): A string used to title the plots.

    Returns:
        None
    """
    # Isolate the desired avalanche
    singleAvalancheData = electronTrackData[electronTrackData['Avalanche ID'] == avalancheID]
    gain = singleAvalancheData['Electron ID'].max()+1
    
    # Group drift lines together based on the electron ID
    groupedAvalancheData = singleAvalancheData.groupby('Electron ID')

    # Create the figure and add subplots
    fig = plt.figure(figsize=(14, 7))
    fig.suptitle(f'Avalanche: {plotName} (ID #{groupedAvalancheData['Avalanche ID'].min().iloc[0]}) Gain = {gain}')
    
    ax11 = fig.add_subplot(221)
    ax12 = fig.add_subplot(223)
    ax13 = fig.add_subplot(122)

    # Iterate through the drift lines of each electron
    for electronID, driftLine in groupedAvalancheData:
        ax11.plot(driftLine['Drift x'], 
                  driftLine['Drift z'], 
                  linewidth=.5)
        ax12.plot(driftLine['Drift y'], 
                  driftLine['Drift z'], 
                  linewidth=.5)
        ax13.plot(driftLine['Drift x'], 
                  driftLine['Drift y'], 
                  linewidth=.5)

    # Plot the initial electron location
    ax11.plot(0., 0.005, label='Initial', marker='x', c='r')
    ax12.plot(0., 0.005, label='Initial', marker='x', c='r')
    ax13.plot(0., 0., label='Initial', marker='x', c='r')
    
    plotAddCellGeometry(ax11, 'xz', metaData)
    plotAddCellGeometry(ax12, 'yz', metaData)
    plotAddCellGeometry(ax13, 'xy', metaData)


    return

In [ ]:
plotSingleAvalanche(electronTrackData, 0, metaData, 'First')

#averageAvalanche = math.floor(avalancheData['Total Electrons'].mean())
#averageAvalancheID = avalancheData[avalancheData['Total Electrons']==averageAvalanche]['Avalanche ID'].iloc[0]
#plotSingleAvalanche(electronTrackData, averageAvalancheID, metaData, 'Average')

largestAvalanche = avalancheData['Total Electrons'].max()
largestAvalancheID = avalancheData[avalancheData['Total Electrons']==largestAvalanche]['Avalanche ID'].iloc[0]
plotSingleAvalanche(electronTrackData, largestAvalancheID, metaData, 'Largest')

In [ ]:
longestZ = -1
longestID = -1
for i in range(len(avalancheData[avalancheData['Total Electrons'] == 1])):
    avalancheID = avalancheData[avalancheData['Total Electrons'] == 1]['Avalanche ID'].iloc[i]

    inAvalanche  = electronData[electronData['Avalanche ID'] == avalancheID]
    x = inAvalanche['Final x'].iloc[0]
    y = inAvalanche['Final y'].iloc[0]
    z = inAvalanche['Final z'].iloc[0]
    stat = inAvalanche['Exit Status'].iloc[0]

    halfPitch = .99*metaData['Pitch'].iloc[0]/2.
    if stat == -7:
        continue
    elif np.abs(x) >= halfPitch or np.abs(y) >= halfPitch:
        if np.abs(z) > longestZ:
            longestID = avalancheID
            longestZ = np.abs(z)

plotSingleAvalanche(electronTrackData, longestID, metaData, 'Furthest Drift')

In [ ]:
def plotAvalancheSize(avalancheData, metaData, binWidth=1) -> None:
    """
    """
    runNo = metaData['runNo'].iloc[0]

    # Make plot for all avalanches
    gain = avalancheData['Total Electrons'].mean()

    bins = np.arange(avalancheData['Total Electrons'].min(), 
                     avalancheData['Total Electrons'].max()+1, 
                     binWidth)

    fig = plt.figure(figsize=(8, 5))
    fig.suptitle(f'All Avalanche Sizes: Run {runNo}')
    
    ax = fig.add_subplot(211)
    
    ax.hist(avalancheData['Total Electrons'], 
             bins=bins, label='Simulation')
    
    ax.axvline(x=gain, 
               c='g', ls='--', label=f'Simulated Gain = {gain:.0f}')
    
    ax.set_xlabel('Number of Electrons')
    ax.set_ylabel('')
    ax.legend()
    ax.grid()

    # Remove avalanches that have 1 electron or reached limit
    trimmedAvalanche = avalancheData[(avalancheData['Total Electrons'] > 1) 
                                    & (avalancheData['Reached Limit'] == 0)]
    
    gain = trimmedAvalanche['Total Electrons'].mean()
    ## Note to self: 
    #     Removing 1e avalanches will increase the average value
    #     Removing the limit-hit ones will decrease the average

    bins = np.arange(trimmedAvalanche['Total Electrons'].min(), 
                     trimmedAvalanche['Total Electrons'].max()+1, 
                     binWidth)

    fig = plt.figure(figsize=(8, 5))
    fig.suptitle(f'Trimmed Avalanche Size: Run {runNo}')
    
    ax = fig.add_subplot(211)
    
    ax.hist(trimmedAvalanche['Total Electrons'], 
             bins=bins, label='Simulation')
    
    ax.axvline(x=gain, 
               c='g', ls='--', label=f'Simulated Gain = {gain:.0f}')
    
    ax.set_xlabel('Number of Electrons')
    ax.set_ylabel('')
    ax.legend()
    ax.grid()
    
    return
    

In [ ]:
plotAvalancheSize(avalancheData, metaData, binWidth=5)

In [ ]:
def polya(n, nBar, theta):
    return (1/nBar)*np.power(theta+1, theta+1)/gamma(theta+1)*np.power(n/nBar, theta)*np.exp(-(n/nBar)*(theta+1))

def exponential(n, nBar):
    #This is a polya with theta=0
    return (1/nBar)*np.exp(-(n/nBar))

def fitPolya(avalancheData, metaData, binWidth=1):
    """
    """

    # Remove avalanches that have 1 electron or reached limit
        ##Note on 1e avalanches:
        ## These certainly can happen, however they are more likely that
        ## the electron has attached or drifted out of the volume.
 
    trimmedAvalanche = avalancheData[(avalancheData['Total Electrons'] > 1) 
                                    & (avalancheData['Reached Limit'] == 0)]

    gain = trimmedAvalanche['Total Electrons'].mean()

    if (gain < 5) or (gain > metaData['Avalanche Limit'].iloc[0]):
        print(f"Unable to do fit. Gain is: {gain:.2f}.")
        return None, None
    
    bins = np.arange(trimmedAvalanche['Total Electrons'].min(), 
                     trimmedAvalanche['Total Electrons'].max()+1, 
                     binWidth)
    binCenters = bins[:-1] + binWidth/2.

    counts, _ = np.histogram(trimmedAvalanche['Total Electrons'], 
                             bins=bins)
    prob = counts/len(trimmedAvalanche['Total Electrons'])/binWidth

    # Get error
    countErr = np.where(counts == 0, 1, np.sqrt(counts))
    probErr = countErr/len(trimmedAvalanche['Total Electrons'])/binWidth

    # Set initial fitting parameters with some bounds
    initial = [gain, 1.]
    bounds = ([1, 0], [metaData['Avalanche Limit'].iloc[0], 10])

    popt, pcov = curve_fit(polya, binCenters, prob, p0=initial, bounds=bounds, sigma=probErr)
    perr = np.sqrt(np.diag(pcov))

    fitPolyaParam = {
        'nBar': popt[0],
        'nBarErr': perr[0],
        'theta': popt[1],
        'thetaErr': perr[1]
    }

    plotParam = {
        'gain': gain,
        'bins': bins,
        'binCenters': binCenters,
        'prob': prob
    }

    return fitPolyaParam, plotParam

def fitExponential(avalancheData, metaData, binWidth=1):
    """
    """

    # Remove avalanches that have 1 electron or reached limit
        ##Note on 1e avalanches:
        ## These certainly can happen, however they are more likely that
        ## the electron has attached or drifted out of the volume.
 
    trimmedAvalanche = avalancheData[(avalancheData['Total Electrons'] > 1) 
                                    & (avalancheData['Reached Limit'] == 0)]

    gain = trimmedAvalanche['Total Electrons'].mean()

    if (gain < 5) or (gain > metaData['Avalanche Limit'].iloc[0]):
        print(f"Unable to do fit. Gain is: {gain:.2f}.")
        return None, None
    
    bins = np.arange(trimmedAvalanche['Total Electrons'].min(), 
                     trimmedAvalanche['Total Electrons'].max()+1, 
                     binWidth)
    binCenters = bins[:-1] + binWidth/2.

    counts, _ = np.histogram(trimmedAvalanche['Total Electrons'], 
                             bins=bins)
    prob = counts/len(trimmedAvalanche['Total Electrons'])/binWidth

    # Get error
    countErr = np.where(counts == 0, 1, np.sqrt(counts))
    probErr = countErr/len(trimmedAvalanche['Total Electrons'])/binWidth

    # Set initial fitting parameters with some bounds
    initial = [gain]
    bounds = ([1], [metaData['Avalanche Limit'].iloc[0]])

    popt, pcov = curve_fit(exponential, binCenters, prob, p0=initial, bounds=bounds, sigma=probErr)
    perr = np.sqrt(np.diag(pcov))

    fitExponentialParam = {
        'nBar': popt[0],
        'nBarErr': perr[0],
    }

    plotParam = {
        'gain': gain,
        'bins': bins,
        'binCenters': binCenters,
        'prob': prob
    }

    return fitExponentialParam, plotParam
    
def plotAvalancheFits(avalancheData, metaData, binWidth=1):
    """
    """
    runNo = metaData['runNo'].iloc[0]
    gainRaw = avalancheData['Total Electrons'].mean()

    fitPolyaParam, plotParam = fitPolya(avalancheData, metaData, binWidth)
    
    polyaFit = polya(plotParam['binCenters'], fitPolyaParam['nBar'], fitPolyaParam['theta'])

    fitExponentialParam, _ = fitExponential(avalancheData, metaData, binWidth)
    
    exponentialFit = exponential(plotParam['binCenters'], fitExponentialParam['nBar'])
    
    fig = plt.figure(figsize=(10, 5))
    fig.suptitle(f'Avalanche Size: Run {runNo}')
    
    ax = fig.add_subplot(111)
    
    ax.bar(plotParam['bins'][:-1], plotParam['prob'], 
           width=binWidth, align='edge', label='Simulated Data', color='b')
    ax.axvline(x=gainRaw, 
               c='g', ls='--', label=f'Simulated Gain (Raw) = {gainRaw:.1f}e')
    ax.axvline(x=plotParam['gain'], 
               c='g', ls=':', label=f'Simulated Gain (Trimmed) = {plotParam['gain']:.1f}e')

    ax.plot(plotParam['binCenters'], polyaFit, 
            'm-', lw=2, label=r'Fitted Polya ($\theta$' + f' = {fitPolyaParam['theta']:.4f})')
    ax.axvline(x=fitPolyaParam['nBar'], 
               c='m', ls=':', label=f'Fitted Gain = {fitPolyaParam['nBar']:.1f}e')

    ax.plot(plotParam['binCenters'], exponentialFit, 
            'r', lw=2, label=f'Fitted Exponential')
    ax.axvline(x=fitExponentialParam['nBar'], 
               c='r', ls=':', label=f'Fitted Gain = {fitExponentialParam['nBar']:.1f}e')
    
    ax.set_xlabel('Number of Electrons')
    ax.set_ylabel('Probability')
    
    ax.legend()
    ax.grid()

    return

In [ ]:
plotAvalancheFits(avalancheData, metaData, binWidth=5)

In [ ]:
def polyaEfficiency(nBar, theta, threshold):
    s = theta+1
    x = s*threshold/nBar
    return gammaincc(s, x)
    
def getPolyaEfficiency(avalancheData, metaData, binWidth=1, threshold=10):
    """
    """

    fitPolyaParam, _ = fitPolya(avalancheData, metaData, binWidth)

    if fitPolyaParam is None:
        print(f"Unable to determine efficiency.")
        return 0, 0

    efficiency = polyaEfficiency(fitPolyaParam['nBar'], fitPolyaParam['theta'], threshold)

    nBarWithErr = [
        fitPolyaParam['nBar'] - fitPolyaParam['nBarErr'],
        fitPolyaParam['nBar'] + fitPolyaParam['nBarErr']
        ]
    thetaWithErr = [
        fitPolyaParam['theta'] - fitPolyaParam['thetaErr'], 
        fitPolyaParam['theta'] + fitPolyaParam['thetaErr']
        ]

    efficiencyWithErrors = np.zeros((2, 2))
    for i, n in enumerate(nBarWithErr):
        for j, t in enumerate(thetaWithErr):
            efficiencyWithErrors[i, j] = polyaEfficiency(n, t, threshold)

    efficiencyErr = [efficiencyWithErrors.max()-efficiency, efficiencyWithErrors.min()-efficiency]
    return efficiency, efficiencyErr

In [ ]:
eff, errs = getPolyaEfficiency(avalancheData, metaData, binWidth=5, threshold=3)
print(f"Efficiency: {eff:.4f}")
print(f"Efficiency Range: {eff + errs[1]:.4f} - {eff + errs[0]:.4f}")

In [ ]:
def plotGeneralPolya():
    """
    """
    n = np.linspace(0, 4, 101)
    theta = np.arange(0, 5, 1)
    plt.figure(figsize=(6, 4))
    
    for t in theta:
        prob = polya(n, 1, t)
        plt.plot(n, prob,
                label=r'$\theta$'+f' = {t:.2f}')
        
    plt.title(f"General Polya Distribution")
    plt.xlabel(r'Avalanche Size ($n/\bar{n}$)')
    plt.ylabel(r'$\bar{n}$ x Probability')
    plt.legend()
    plt.grid(True, alpha=0.5)
    plt.show()

def plotPolya(theta):
    """
    """
    gain = [10, 25, 50, 75, 100]

    n = np.arange(0, 101, 1)
    
    numPlots = len(theta)
    numRows = int(np.ceil(numPlots/2))
    
    # Create the figure and add subplots
    fig, axes = plt.subplots(nrows=numRows, ncols=2, figsize=(12, 5*numRows))
    axesFlat = axes.flatten()

    fig.suptitle(f'Polya Avalanches')

    for i, t in enumerate(theta):
        for nBar in gain:
            prob = polya(n, nBar, t)
            axesFlat[i].plot(n, prob,
                             label=r'$\bar{n}$'+f' = {nBar:.0f}')

        axesFlat[i].set_title(r'$\theta$'+f' = {t:0.2f}')
        axesFlat[i].set_xlabel('Avalanche size')
        axesFlat[i].set_ylabel('Probability')
        axesFlat[i].legend()
        axesFlat[i].grid(True, alpha=0.5)

    for j in range(numPlots, len(axesFlat)):
        fig.delaxes(axesFlat[j])
    plt.show()

def plotPolyaEfficiency(theta):
    """
    """
    k = np.linspace(0, 1, 101) #Ratio: Threshold/Gain

    plt.figure(figsize=(12, 5))

    for t in theta:
        efficiency = gammaincc(t+1, k*(t+1))
        plt.plot(k, efficiency,
                 label=r"$\theta$"+f" = {t:0.2f}")

    targetEfficiency = 0.95
    plt.axhline(y=targetEfficiency,
                c='r', ls='--', label=f'{targetEfficiency*100:.0f}% Efficiency')
    plt.axvline(x=-np.log(targetEfficiency),
                c='r', ls=':', label=r'$\theta = 0$ Max: '+f'{-np.log(targetEfficiency):.3f}')

    plt.title(f"Efficiency of Polya: "
              +r"$\eta = \frac{\Gamma\left(\theta+1, (\theta+1)*n_{t}/\bar{n}\right)}{\Gamma\left(\theta+1\right)}$")
    plt.xlabel("Threshold / Gain Fraction: "
               +r"$n_{t} / \bar{n}$")
    plt.ylabel(f"Efficiency")
    plt.legend()
    plt.grid(True, alpha=0.5)
    plt.show()

    return

def solvePolyEfficiency(nBar, theta, threshold, targetEff):

    calcEff = polyaEfficiency(nBar, theta, threshold)
    return targetEff - calcEff
    
def solveForGain(efficiency, theta, threshold):
    """
    """
    gainMin = 1
    gainSolved, _, _, mesg = fsolve(solvePolyEfficiency,
                                    gainMin, args=(theta, threshold, efficiency), full_output=True)
    #print(f'Gain Solution: {gainSolved[0]:.4f} ({mesg})')

    return gainSolved[0]

def plotThreshold():
    """
    """
    threshold = np.linspace(0, 16, 11)
    efficiency = [.95, .9]

    colors = ['b', 'r', 'g']

    plt.figure(figsize=(6, 4))
    
    for i, eff in enumerate(efficiency):
        gain = -threshold/np.log(eff)
        plt.plot(threshold, gain,
                 c=colors[i], label=f'Target Efficiency = {eff*100:.0f}%')

        theta5 = threshold*solveForGain(eff, 0.5, 1)
        plt.plot(threshold, theta5,
                 c=colors[i], ls=':', label=r'$\theta$ = 0.5')
        
        theta1 = threshold*solveForGain(eff, 1, 1)
        plt.plot(threshold, theta1,
                 c=colors[i], ls='--', label=r'$\theta$ = 1')

        theta2 = threshold*solveForGain(eff, 2, 1)
        plt.plot(threshold, theta2,
                 c=colors[i], ls='-.', label=r'$\theta$ = 2')

    plt.title(f"Minimum Gain Required to Achieve Efficiency")
    plt.xlabel('Detector Threshold')
    plt.ylabel('Gain')
    plt.legend()
    plt.grid(True, alpha=0.5)
    plt.show()


In [ ]:
def plotPolyExamples(thetaStart, thetaEnd, numSteps):
    """
    """
    theta = np.linspace(thetaStart, thetaEnd, numSteps)

    plotGeneralPolya()
    plotPolya(theta)
    plotPolyaEfficiency(theta)
    plotThreshold()
    
plotPolyExamples(0, 5, 6)

In [ ]:
def readMultipleRuns(runNumbers, treesToRead):
    """
    """
    allMetaData = {}
    allData = {}
    
    for inRun in runNumbers:
        
        simData = readRootTrees(inRun)
        
        allMetaData[f'{inRun}'] = simData['metaDataTree;1']
        allData[f'{inRun}'] = {}

        for inTree in treesToRead:
            allData[f'{inRun}'][inTree] = simData[f'{inTree}Tree;1']

    return allMetaData, allData

In [ ]:
def plotScannedPolyaParam(runNumbers):
    """
    """
    metaData, allData = readMultipleRuns(runNumbers, ['avalancheData'])


    holeRadius = []
    standoff = []
    polyaGain = []
    polyaTheta = []
    polyaGainErr = []
    polyaThetaErr = []
    
    for inRun in runNumbers:
        fitPolyaParam, _ = fitPolya(allData[f'{inRun}']['avalancheData'], metaData[f'{inRun}'])

        holeRadius.append(metaData[f'{inRun}']['Hole Radius'].iloc[0]*CMTOMICRON)
        standoff.append(metaData[f'{inRun}']['Grid Standoff'].iloc[0]*CMTOMICRON)
        polyaGain.append(fitPolyaParam['nBar'])
        polyaGainErr.append(fitPolyaParam['nBarErr'])
        polyaTheta.append(fitPolyaParam['theta'])
        polyaThetaErr.append(fitPolyaParam['thetaErr'])
        

    fig3D = plt.figure(figsize=(12, 6))
    fig3D.suptitle(f'Test')
    
    ax3D1 = fig3D.add_subplot(121, projection='3d')
    ax3D2 = fig3D.add_subplot(122, projection='3d')

    ax3D1.errorbar(holeRadius, standoff, polyaGain,
                 zerr=polyaGainErr, ls='', marker='o')
    ax3D2.errorbar(holeRadius, standoff, polyaTheta,
                 zerr=polyaThetaErr, ls='', marker='o')

    ax3D1.set_xlabel('Hole Radius (um)')
    ax3D1.set_ylabel('Standoff (um)')
    ax3D1.set_zlabel('Polya Gain')

    ax3D2.set_xlabel('Hole Radius (um)')
    ax3D2.set_ylabel('Standoff (um)')
    ax3D2.set_zlabel('Polya Theta')


    return

#trialARuns = list(range(5209, 5218 + 1))
#plotScannedPolyaParam(trialARuns)